In [ ]:
# !pip install evaluate

In [1]:
import numpy as np
import random
from transformers import ResNetModel
from torch import nn
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import v2
import torch
import pandas as pd
import evaluate

# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
import numpy as np
from transformers import ResNetModel
from torch import nn
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import v2
import torch
import pandas as pd
import evaluate
from torch.utils.data import DataLoader
import torch.optim as optim

In [3]:
# DEVICE = 'cuda'
DEVICE = torch.device("cuda")

base_path = 'archive/'
img_path = f'{base_path}Food Images/Food Images/'
cap_path = f'{base_path}Food Ingredients and Recipe Dataset with Image Name Mapping.csv'

data = pd.read_csv(cap_path)
partitions = np.load("datasets/Food_Images/food_partitions.npy", allow_pickle=True).item()
print(len(partitions["train"]))  # Access train partition
print(len(partitions["test"]))   # Access test partition
print(len(partitions["valid"])) 

10800
1351
1350


In [4]:
dropped_indices = data[data["Title"].isna()].index  # Get indices of dropped rows
partitions['train'] = [idx for idx in partitions['train'] if idx not in dropped_indices]
print(len(partitions['train']))

10797


In [5]:
data = data.dropna(subset=["Title"])
len(data)

13496

In [6]:
import os
image_folder = f"archive/Food Images/Food Images"
# valid_images = list(set(os.listdir(image_folder)))

valid_images = list({os.path.splitext(f)[0] for f in os.listdir(image_folder)})

print(len(valid_images))
# print(valid_images[:5])
# print(data["Image_Name"].head())
      
data = data[data["Image_Name"].isin(valid_images)]

# Reset index after filtering
data = data.reset_index(drop=True)

13582


In [7]:
valid_indices = set(data.index)  # These are the indices that remain after filtering

# Step 2: Filter `partitions['train']` to keep only valid indices
partitions['train'] = [idx for idx in partitions['train'] if idx in valid_indices]
print(len(partitions['train']))
print(len(data))

10767
13466


In [8]:
print(len(partitions['train']))
print(len(data))

10767
13466


In [9]:
max_index = max(partitions['train'])
print("Max index:", max_index)

Max index: 13465


In [10]:
unique_chars_1 = set("".join(data["Ingredients"].astype(str)))
unique_chars_2 = set("".join(data["Instructions"].astype(str)))
unique_chars_3 = set("".join(data["Image_Name"].astype(str)))
unique_chars_4 = set("".join(data["Cleaned_Ingredients"].astype(str)))


print(len(unique_chars_1))
print(len(unique_chars_2))
print(len(unique_chars_3))
print(len(unique_chars_4))

148
174
37
147


In [11]:
import unicodedata

# Normalize and remove unwanted characters
def clean_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalize Unicode
    text = text.encode("ascii", "ignore").decode("ascii")  # Remove non-ASCII chars
    return text

# Apply cleaning to the Title column
data["Title"] = data["Title"].astype(str).apply(clean_text)

# Extract unique characters
chars = list(set("".join(data["Title"])))

# Ensure special tokens are first
chars = ['<SOS>', '<EOS>', '<PAD>'] + sorted(chars)

In [12]:
# # Extract unique characters from the dataset
# chars = list(set("".join(data["Title"].astype(str))))

# # Sort the extracted characters
# chars.sort()

# # Prepend special tokens
# chars = ['<SOS>', '<EOS>', '<PAD>'] + chars

# # Print the final ordered list
# # print(chars)

In [13]:
# chars = ['<SOS>', '<EOS>', '<PAD>', ' ', '!', '"', '#', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
# chars = list(set("".join(data["Title"].astype(str))))
# chars.append('<SOS>')
# chars.append('<EOS>')
# chars.append('<PAD>')
# global NUM_CHAR
 
NUM_CHAR = len(chars)
idx2char = {k: v for k, v in enumerate(chars)}
char2idx = {v: k for k, v in enumerate(chars)}

TEXT_MAX_LEN = 201

In [14]:
class Data(Dataset):
    def __init__(self, data, partition):
        self.data = data
        self.partition = partition
        self.num_captions = 5
        self.max_len = TEXT_MAX_LEN
        self.img_proc = torch.nn.Sequential(
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Resize((224, 224), antialias=True),
            v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),)

    def __len__(self):
        return len(self.partition)
    
    def __getitem__(self, idx):
        real_idx = self.partition[idx]  # Row index in dataset
        item = self.data.iloc[real_idx]  # Get row
                
        img_name = item.Image_Name + '.jpg'
        # print(img_name)
        img = Image.open(f'{img_path}{img_name}').convert('RGB')
        img = self.img_proc(img)
        
        caption = item["Title"]
        cap_list = list(caption)

        final_list = [chars[0]]
        final_list.extend(cap_list)
        final_list.extend([chars[1]])
        gap = self.max_len - len(final_list)
        final_list.extend([chars[2]]*gap)

        missing_chars = [c for c in final_list if c not in char2idx]
        if missing_chars:
            print(f"Missing characters: {set(missing_chars)}")

        for char in missing_chars:
            if char not in char2idx:
                char2idx[char] = len(char2idx)  # Assign a new index

        cap_idx = [char2idx[i] for i in final_list]

        # return img, cap_idx
        return img, torch.tensor(cap_idx, dtype=torch.long)

In [15]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = ResNetModel.from_pretrained('microsoft/resnet-18').to(DEVICE)
        self.gru = nn.GRU(512, 512, num_layers=1)
        self.proj = nn.Linear(512, NUM_CHAR)
        self.embed = nn.Embedding(NUM_CHAR, 512)
        self.num_classes = NUM_CHAR
        
    def forward(self, img):
        batch_size = img.shape[0]
        feat = self.resnet(img)
        feat = feat.pooler_output.squeeze(-1).squeeze(-1).unsqueeze(0) # 1, batch, 512
        start = torch.tensor(char2idx['<SOS>']).to(DEVICE)
        start_embed = self.embed(start) # 512
        start_embeds = start_embed.repeat(batch_size, 1).unsqueeze(0) # 1, batch, 512
        inp = start_embeds
        hidden = feat
        for t in range(TEXT_MAX_LEN-1): # rm <SOS>
            out, hidden = self.gru(inp, hidden)
            inp = torch.cat((inp, out[-1:]), dim=0) # N, batch, 512
    
        res = inp.permute(1, 0, 2) # batch, seq, 512
        res = self.proj(res) # batch, seq, 80
        res = res.permute(0, 2, 1) # batch, 80, seq
        return res

In [16]:
max(partitions['train'])

13465

In [17]:
NUM_CHAR

81

In [18]:
'''A simple example to calculate loss of a single batch (size 2)'''
dataset = Data(data, partitions['train'])

img1, caption1 = next(iter(dataset))

img2, caption2 = next(iter(dataset))

caption1 = torch.tensor(caption1)
caption2 = torch.tensor(caption2)
img = torch.cat((img1.unsqueeze(0), img2.unsqueeze(0)))
caption = torch.cat((caption1.unsqueeze(0), caption2.unsqueeze(0)))
img, caption = img.to(DEVICE), caption.to(DEVICE)
model = Model().to(DEVICE)
pred = model(img)
crit = nn.CrossEntropyLoss()
loss = crit(pred, caption)
print(loss)


C:\Users\Dell\AppData\Local\Temp\ipykernel_19764\108332190.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  caption1 = torch.tensor(caption1)
C:\Users\Dell\AppData\Local\Temp\ipykernel_19764\108332190.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  caption2 = torch.tensor(caption2)


tensor(4.4490, device='cuda:0', grad_fn=<NllLoss2DBackward0>)


In [19]:
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Current Device: 0
Device Name: NVIDIA GeForce GTX 1660 Ti with Max-Q Design


In [20]:
import torch

# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()
# torch.cuda.reset_max_memory_allocated()
# torch.cuda.reset_max_memory_cached()
# torch.cuda.synchronize()

DEVICE = torch.device("cuda")
print("Using device:", DEVICE)

try:
    x = torch.rand(3, 3).to(DEVICE)
    print("Tensor successfully moved to:", x.device)
except Exception as e:
    print("Error:", e)


Using device: cuda
Tensor successfully moved to: cuda:0


In [21]:
'''metrics'''
bleu = evaluate.load('bleu')
meteor = evaluate.load('meteor')
rouge = evaluate.load('rouge')

reference = [['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .']]
prediction = ['A girl goes into a wooden building .']

res_b = bleu.compute(predictions=prediction, references=reference)
res_r = rouge.compute(predictions=prediction, references=reference)
res_m = meteor.compute(predictions=prediction, references=reference)

res_b, res_r, res_m

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


({'bleu': 0.5946035575013605,
  'precisions': [0.875, 0.7142857142857143, 0.5, 0.4],
  'brevity_penalty': 1.0,
  'length_ratio': 1.0,
  'translation_length': 8,
  'reference_length': 8},
 {'rouge1': 0.8571428571428571,
  'rouge2': 0.6666666666666666,
  'rougeL': 0.8571428571428571,
  'rougeLsum': 0.8571428571428571},
 {'meteor': 0.864795918367347})

In [22]:
ref = [['A child is running in the campus']]
pred1 = ['A child is running']

res_b = bleu.compute(predictions=pred1, references=ref)
res_r = rouge.compute(predictions=pred1, references=ref)
res_m = meteor.compute(predictions=pred1, references=ref)

res_b, res_r, res_m

({'bleu': 0.4723665527410147,
  'precisions': [1.0, 1.0, 1.0, 1.0],
  'brevity_penalty': 0.4723665527410147,
  'length_ratio': 0.5714285714285714,
  'translation_length': 4,
  'reference_length': 7},
 {'rouge1': 0.7272727272727273,
  'rouge2': 0.6666666666666666,
  'rougeL': 0.7272727272727273,
  'rougeLsum': 0.7272727272727273},
 {'meteor': 0.5923507462686567})

In [23]:
ref = [['A child is running in the campus']]
pred1 = ['A child is']

res_b = bleu.compute(predictions=pred1, references=ref)
res_r = rouge.compute(predictions=pred1, references=ref)
res_m = meteor.compute(predictions=pred1, references=ref)

res_b, res_r, res_m

({'bleu': 0.0,
  'precisions': [1.0, 1.0, 1.0, 0.0],
  'brevity_penalty': 0.2635971381157267,
  'length_ratio': 0.42857142857142855,
  'translation_length': 3,
  'reference_length': 7},
 {'rouge1': 0.6, 'rouge2': 0.5, 'rougeL': 0.6, 'rougeLsum': 0.6},
 {'meteor': 0.44612794612794615})

In [24]:
ref = [['A child is running in the campus']]
pred1 = ['A child campus']

res_b = bleu.compute(predictions=pred1, references=ref)
res_r = rouge.compute(predictions=pred1, references=ref)
res_m = meteor.compute(predictions=pred1, references=ref)
res_m_sin = meteor.compute(predictions=pred1, references=ref, gamma=0) # no penalty by setting gamma to 0

res_b, res_r, res_m, res_m_sin

({'bleu': 0.0,
  'precisions': [1.0, 0.5, 0.0, 0.0],
  'brevity_penalty': 0.2635971381157267,
  'length_ratio': 0.42857142857142855,
  'translation_length': 3,
  'reference_length': 7},
 {'rouge1': 0.6, 'rouge2': 0.25, 'rougeL': 0.6, 'rougeLsum': 0.6},
 {'meteor': 0.3872053872053872},
 {'meteor': 0.45454545454545453})

Final metric we use for challenge 3: BLEU1, BLEU2, ROUGE-L, METEOR

In [25]:
ref = [['A child is running in the campus']]
pred1 = ['A child campus']

bleu1 = bleu.compute(predictions=pred1, references=ref, max_order=1)
bleu2 = bleu.compute(predictions=pred1, references=ref, max_order=2)
res_r = rouge.compute(predictions=pred1, references=ref)
res_m = meteor.compute(predictions=pred1, references=ref)

f"BLEU-1:{bleu1['bleu']*100:.1f}%, BLEU2:{bleu2['bleu']*100:.1f}%, ROUGE-L:{res_r['rougeL']*100:.1f}%, METEOR:{res_m['meteor']*100:.1f}%"

'BLEU-1:26.4%, BLEU2:18.6%, ROUGE-L:60.0%, METEOR:38.7%'

Now it is your turn! Try to finish the code below to run the train function

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

def decode_caption(indices, vocab):
    """Convert tensor indices to text using the character list"""
    return ''.join([vocab[idx] if idx < len(vocab) else '<UNK>' for idx in indices if idx not in [0]])  # Remove <PAD>


def train(EPOCHS, batch_size=4):
    # Create datasets
    data_train = Data(data, partitions['train'])
    data_valid = Data(data, partitions['valid'])
    data_test = Data(data, partitions['test'])
    
    # Define DataLoaders
    dataloader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=0)
    dataloader_valid = DataLoader(data_valid, batch_size=batch_size, shuffle=False, num_workers=0)
    dataloader_test = DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=0)

    
    print("DataLoader process is finished")
    # Initialize model, optimizer, and loss function
    model = Model().to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    crit = nn.CrossEntropyLoss()

    # Training loop
    os.makedirs("week3/models", exist_ok=True)

    for epoch in range(EPOCHS):
        print(f"Starting the process of the epoch {epoch}")
        model.train()
        train_loss, train_acc = train_one_epoch(model, optimizer, crit, dataloader_train)
        print(f'Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')

        checkpoint_path = f"models/model_weights_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)

        # Validation step   
        # model.eval()
        # valid_loss, valid_acc = eval_epoch(model, crit, dataloader_valid)
        # print(f'Validation Loss: {valid_loss:.4f}, Validation Acc: {valid_acc:.4f}')

        print(f"Model weights saved to {checkpoint_path}")

    # Final evaluation on test set
    # test_loss, test_acc = eval_epoch(model, crit, dataloader_test)
    # print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

def train_one_epoch(model, optimizer, crit, dataloader):

    model.train()
    total_loss = 0
    bleu1_score, bleu2_score, rouge_score, meteor_score = 0, 0, 0, 0

    for imgs, captions in dataloader:
        imgs, captions = imgs.to(DEVICE), captions.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)  # Forward pass

        captions = captions.long()  # Ensure correct dtype

        loss = crit(outputs, captions)  # Compute loss

        loss.backward()  # Backpropagation
        optimizer.step()
        
        total_loss += loss.item()
        
        _, predicted = outputs.max(1)

        decoded_refs = [decode_caption(caption.cpu().numpy(), chars) for caption in captions]
        decoded_preds = [decode_caption(pred.cpu().numpy(), chars) for pred in predicted]
        
        print(f"Ref: {decoded_refs}")
        print(f"Pred: {decoded_preds}")

        bleu1 = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_refs], max_order=1)
        bleu2 = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_refs], max_order=2)
        res_r = rouge.compute(predictions=decoded_preds, references=decoded_refs)
        res_m = meteor.compute(predictions=decoded_preds, references=decoded_refs)

        # Accumulate scores
        bleu1_score += bleu1["bleu"]
        bleu2_score += bleu2["bleu"]
        rouge_score += res_r["rougeL"]
        meteor_score += res_m["meteor"]

        print(f"BLEU-1: {bleu1['bleu']:.4f}, BLEU-2: {bleu2['bleu']:.4f}, ROUGE-L: {res_r['rougeL']:.4f}, METEOR: {res_m['meteor']:.4f}")
        
        print("_" * 80)

        # _, predicted = outputs.max(2)
        # correct = 0
        # correct += (predicted == captions).sum().item()

    # Compute averages
    avg_loss = total_loss / len(dataloader)
    bleu1_score /= len(dataloader)
    bleu2_score /= len(dataloader)
    rouge_score /= len(dataloader)
    meteor_score /= len(dataloader)

    print(f"BLEU-1: {bleu1_score:.4f}, BLEU-2: {bleu2_score:.4f}, ROUGE-L: {rouge_score:.4f}, METEOR: {meteor_score:.4f}")
    
    avg_loss = total_loss / len(dataloader)
    # accuracy = 100 * correct / total
    accuracy = 100
    return avg_loss, accuracy

def eval_epoch(model, crit, dataloader):
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for imgs, captions in dataloader:
            imgs, captions = imgs.to(DEVICE), captions.to(DEVICE)

            outputs = model(imgs)
            loss = crit(outputs, captions)

            total_loss += loss.item() * imgs.size(0)
            total_correct += (outputs.argmax(dim=1) == captions).sum().item()
            total_samples += imgs.size(0)

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples
    return avg_loss, avg_acc


In [ ]:
train(10)

DataLoader process is finished
Starting the process of the epoch 0
Ref: ['Short Ribs Slow-Roasted in Coconut Milk<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>', 'Gambas al Ajillo<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><P

In [ ]:
# data["Title"] = data["Title"].astype(str).apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
# xer = list(set("".join(data["Title"].astype(str))))

# print(xer)